<a href="https://colab.research.google.com/github/jmcnerney1/Jack-Mcnerney-Python/blob/main/Jack-famafrench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm


factors = pd.read_csv('data/fama_french_factors.csv', index_col=0, parse_dates=True)
factors = factors / 100


tickers = ['AAPL', 'MSFT']
start = '2018-01-01'
end = '2023-12-31'

results = []

for ticker in tickers:
    stock = yf.download(ticker, start=start, end=end, interval='1mo')['Adj Close'].dropna()
    stock_ret = stock.pct_change().dropna()

    aligned = pd.concat([stock_ret, factors], axis=1, join='inner')
    aligned.columns = ['stock_return', 'MKT_RF', 'SMB', 'HML', 'RF']

    aligned['excess_return'] = aligned['stock_return'] - aligned['RF']

    X = aligned[['MKT_RF', 'SMB', 'HML']]
    y = aligned['excess_return']
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    alpha = model.params['const']
    betas = model.params[['MKT_RF', 'SMB', 'HML']]


    avg_factors = aligned[['MKT_RF', 'SMB', 'HML']].mean()
    rf_avg = aligned['RF'].mean()
    cost_of_equity = rf_avg + np.dot(betas, avg_factors)

    sharpe_ratio = aligned['excess_return'].mean() / aligned['excess_return'].std()

    results.append({
        'Ticker': ticker,
        'Alpha': alpha,
        'Betas': betas.to_dict(),
        'Cost of Equity': cost_of_equity,
        'Sharpe Ratio': sharpe_ratio
    })


df = pd.DataFrame(results)
print(df.to_string(index=False))

FileNotFoundError: [Errno 2] No such file or directory: 'data/fama_french_factors.csv'